# Set Up

In [1]:
# import necessary packages
import pandas as pd
import numpy as np
import os

In [3]:
# flag to save CSVs
save_csv = True

# set directories
base_dir = f'{os.path.dirname(os.getcwd())}/'
csv_dir = f'{base_dir}analysis/CSVs/'

# Combine group and individual roi PSC

In [6]:
# load csvs
group_psc = pd.read_csv(f'{csv_dir}groupROIs_allDataMaps_meanPSC.csv')
native_psc = pd.read_csv(f'{csv_dir}allDataROIs_allDataMaps_meanPSC.csv')

# filter data
native_psc=native_psc.query('space=="native"')
native_psc = native_psc[['sub','roi','category','mean_psc']]

# combine data
combined_data = pd.concat([group_psc,native_psc])

# filter data
combined_data = combined_data.query('roi=="VWFA" or roi=="cVWFA" or roi=="aVWFA"or roi=="rVWFA" or roi=="kVWFA"')

# save csv
if save_csv:
    combined_data.to_csv(f'{csv_dir}native-group-VWFAs_PSC.csv',index=False)

# Calculate selectivity index

In [8]:
# load the psc data
psc_df = pd.read_csv(f'{csv_dir}native-group-VWFAs_PSC.csv')

#filter fo only categories of interest
psc_data = psc_df.query('category=="T" or category=="noText"')

# pivot wider for convience of SI calculation
pivoted_df = psc_data.pivot(columns=['category'],values=['mean_psc'],index=['sub','roi'])
pivoted_df=pivoted_df.reset_index()

# reanme columns
pivoted_df.columns = ['_'.join(filter(None, col)).strip() for col in pivoted_df.columns]
pivoted_df = pivoted_df.rename_axis(None, axis=1)

# calcualte SI
pivoted_df['SI'] = ((1 + pivoted_df['mean_psc_T']) - (1 + pivoted_df['mean_psc_noText'])) / ((1 + pivoted_df['mean_psc_T']) + (1 + pivoted_df['mean_psc_noText']))

#save df
if save_csv:
    pivoted_df.to_csv(f'{csv_dir}groupVWFAs_SI.csv',index=False)